In [1]:
!pip install --upgrade pip
!pip install streamlit pandas numpy matplotlib seaborn scipy wordcloud streamlit-folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [streamlit-folium]


In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [konlpy]


In [3]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import HeatMap
from PIL import Image
import matplotlib.font_manager as fm
from matplotlib.ticker import FuncFormatter
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# --- Colab Environment Setup for Korean Fonts ---
# These commands install Nanum Gothic font and refresh font cache in Colab.
# They need to be run once in the Colab environment.
# Check if font is already installed to avoid unnecessary re-installation
try:
    fm.findfont('NanumGothic', fallback_to_default=False)
    print("NanumGothic font already installed.")
except ValueError:
    print("Installing NanumGothic font...")
    # These commands run on the Colab Linux environment
    !apt-get update -qq
    !apt-get install fonts-nanum -qq
    !fc-cache -fv
    !rm -rf ~/.cache/matplotlib

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

2025-07-07 00:16:13.427 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Installing NanumGothic font...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.

In [13]:
%%writefile app.py

import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static, st_folium
from folium import plugins
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import matplotlib.font_manager as fm
from matplotlib.ticker import FuncFormatter
from scipy.stats import pearsonr
from konlpy.tag import Okt
from collections import Counter
import time
import subprocess # This line should be here
import matplotlib.patches as patches
import random
from datetime import datetime, timedelta # datetime과 timedelta를 임포트합니다.


# --- 페이지 설정 ---
st.set_page_config(
    page_title="전기차 인프라 대시보드",
    page_icon="🔌",
    layout="wide"
)

# --- Colab Environment Setup for Korean Fonts (Moved to main for better control) ---
# This part is now handled within the main() function to ensure it runs once.

# --- Data Loading and Preprocessing ---
@st.cache_data
def load_common_data():
    """공통 데이터 로드 및 전처리"""
    car_path = "ele_car_now_half.csv"
    charger_path = "ele_car_charger_final.csv"
    allcar_path = "var_car.csv"
    crawling_path = "ele_crawling.csv"
    stopwords_path = "stopwords1.txt"

    data_dict = {}

    try:
        data_dict['df_car'] = pd.read_csv(car_path, encoding='UTF-8')
        data_dict['df_charger'] = pd.read_csv(charger_path, encoding='UTF-8')
        data_dict['df_allcar'] = pd.read_csv(allcar_path, encoding='UTF-8')
        data_dict['df_crawling'] = pd.read_csv(crawling_path, encoding='cp949')
    except FileNotFoundError as e:
        st.error(f"에러: 필요한 CSV 파일을 찾을 수 없습니다. ({e})")
        st.markdown(f"**다음 파일들이 Colab 환경에 업로드되었는지 확인해주세요:**")
        st.markdown(f"- `{car_path}`")
        st.markdown(f"- `{charger_path}`")
        st.markdown(f"- `{allcar_path}`")
        st.markdown(f"- `{crawling_path}`")
        st.markdown(f"- `{stopwords_path}` (워드클라우드용)")
        st.stop()
    except Exception as e:
        st.error(f"데이터 로드 중 오류 발생: {e}")
        st.stop()

    for df_name in ['df_car', 'df_charger', 'df_allcar']:
        if df_name in data_dict and 'full_date' in data_dict[df_name].columns:
            data_dict[df_name]['full_date'] = pd.to_datetime(data_dict[df_name]['full_date'])

    df_ev_chg_merge = pd.merge(data_dict['df_car'][['name', 'full_date', 'year', 'car']],
                                 data_dict['df_charger'][['name', 'full_date', 'year', 'total']],
                                 on=['name', 'full_date', 'year'])
    df_ev_chg_merge['ev_per_charger'] = df_ev_chg_merge['car'] / df_ev_chg_merge['total']
    df_ev_chg_merge = df_ev_chg_merge.replace([np.inf, -np.inf], np.nan).dropna(subset=['ev_per_charger'])

    latest_year_data = df_ev_chg_merge[df_ev_chg_merge['year'] == df_ev_chg_merge['year'].max()]

    regional_summary = latest_year_data.groupby('name').agg(
        ev_counts=('car', 'sum'),
        charger_counts=('total', 'sum'),
        avg_ev_per_charger=('ev_per_charger', 'mean')
    ).reset_index()

    regions = regional_summary['name'].tolist()
    ev_counts = regional_summary['ev_counts'].tolist()
    charger_counts = regional_summary['charger_counts'].tolist()

    colors = plt.cm.tab20.colors[:len(regions)]

    car_total_yearly = data_dict['df_car'].groupby('year')['car'].sum().reset_index()
    charger_total_yearly = data_dict['df_charger'].groupby('year')['total'].sum().reset_index()

    df_total_yearly = pd.merge(car_total_yearly, charger_total_yearly, on='year')
    df_total_yearly['car_thousands'] = df_total_yearly['car'] / 1000
    df_total_yearly['charger_thousands'] = df_total_yearly['total'] / 1000

    years = df_total_yearly['year'].astype(str).tolist()
    total_ev_yearly = df_total_yearly['car_thousands'].tolist()
    total_charger_yearly = df_total_yearly['charger_thousands'].tolist()

    df_ev_allcar_merge = pd.merge(data_dict['df_car'][['name', 'full_date', 'year', 'car']],
                                  data_dict['df_allcar'][['name', 'full_date', 'year', 'allcar']],
                                  on=['name', 'full_date', 'year'])
    df_ev_allcar_merge['ev_ratio'] = df_ev_allcar_merge['car'] / df_ev_allcar_merge['allcar'] * 100
    df_ev_allcar_merge = df_ev_allcar_merge.replace([np.inf, -np.inf], np.nan).dropna(subset=['ev_ratio'])

    all_text = ' '.join(data_dict['df_crawling'].iloc[:, 0].dropna().astype(str))
    okt = Okt()
    nouns = okt.nouns(all_text)

    stopwords = set()
    try:
        with open(stopwords_path, 'r', encoding='utf-8') as f:
            stopwords = set(word.strip() for word in f if word.strip())
    except FileNotFoundError:
        st.warning(f"Stopwords file '{stopwords_path}' not found. Word cloud will not filter stopwords.")
    except Exception as e:
        st.warning(f"Error loading stopwords: {e}. Word cloud will not filter stopwords.")

    filtered_nouns = [noun for noun in nouns if len(noun) > 1 and noun not in stopwords]
    word_count = Counter(filtered_nouns)

    data_dict.update({
        'regional_summary': regional_summary,
        'regions': regions,
        'ev_counts': ev_counts,
        'charger_counts': charger_counts,
        'colors': colors,
        'years': years,
        'total_ev_yearly': total_ev_yearly,
        'total_charger_yearly': total_charger_yearly,
        'df_ev_chg_merge': df_ev_chg_merge,
        'df_ev_allcar_merge': df_ev_allcar_merge,
        'word_count': word_count
    })
    return data_dict

@st.cache_data(ttl=3600)
def load_charger_location_data(file_path='/content/전기차충전소2.xlsx'):
    """
    충전소 위치 데이터 (위도, 경도)를 로드하고 전처리하는 함수
    """
    start_time = time.time()
    try:
        df = pd.read_excel(file_path, engine='openpyxl', dtype_backend='pyarrow')

        required_columns = ['위도경도', '충전소명', '주소', '시도', '군구', '설치년도']
        existing_columns = [col for col in required_columns if col in df.columns]
        df = df[existing_columns]

        if '위도경도' in df.columns:
            df[['위도', '경도']] = df['위도경도'].str.split(',', expand=True)
            df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
            df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
        else:
            st.error("Error: '위도경도' 컬럼을 Excel 파일에서 찾을 수 없습니다.")
            return pd.DataFrame()

        df['설치년도'] = pd.to_numeric(df['설치년도'], errors='coerce').astype('Int64')

        df_clean = df.dropna(subset=['위도', '경도', '설치년도'])

        df_clean = df_clean[
            (df_clean['위도'] >= 33.0) & (df_clean['위도'] <= 39.0) &
            (df_clean['경도'] >= 124.0) & (df_clean['경도'] <= 132.0)
        ].reset_index(drop=True)

        load_time = time.time() - start_time
        #st.success(f"충전소 위치 데이터 로드 완료 ({load_time:.2f}초, {len(df_clean):,}개 충전소)")
        return df_clean

    except FileNotFoundError:
        st.error(f"Error: '{file_path}' 파일을 찾을 수 없습니다. Colab 환경에 업로드되었는지 확인해주세요.")
        return pd.DataFrame()
    except Exception as e:
        st.error(f"충전소 위치 데이터 로드 오류: {e}")
        return pd.DataFrame()


# --- 사이드바 메뉴 ---
def render_sidebar():
    """사이드바 메뉴 렌더링"""
    st.sidebar.title("카테고리")

    # 간단한 CSS
    st.markdown("""
      <style>
          /* 사이드바 컨테이너 패딩 조정 */
          section[data-testid="stSidebar"] > div {
              padding-left: 1rem;
              padding-right: 1rem;
          }

          /* 사이드바 버튼 전체 너비 */
          section[data-testid="stSidebar"] .stButton {
              width: 100% !important;
          }

          section[data-testid="stSidebar"] .stButton > button {
              width: 100% !important;
              text-align: left !important;
              justify-content: flex-start !important;
              padding: 0.5rem 1rem !important;
              margin: 0.25rem 0 !important;
              min-height: 2.5rem !important;
              border-radius: 0.5rem !important;
              background-color: #f0f2f6 !important;
              border: 1px solid transparent !important;
              transition: all 0.2s ease !important;
          }

          /* 버튼 호버 효과 */
          section[data-testid="stSidebar"] .stButton > button:hover {
              background-color: #e6f3ff !important;
              border-left: 3px solid #4ECDC4 !important;
              padding-left: calc(1rem - 2px) !important;
              transform: translateX(2px);
          }

          /* 버튼 클릭(활성) 상태 */
          section[data-testid="stSidebar"] .stButton > button:active,
          section[data-testid="stSidebar"] .stButton > button:focus {
              background-color: #4ECDC4 !important;
              color: white !important;
              box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
              outline: none !important;
          }

          /* 버튼 내부 텍스트 */
          section[data-testid="stSidebar"] .stButton > button > div {
              text-align: left !important;
              width: 100% !important;
          }

          section[data-testid="stSidebar"] .stButton > button p {
              text-align: left !important;
              margin: 0 !important;
              font-weight: 500 !important;
          }

          /* 사이드바 제목 스타일 */
          section[data-testid="stSidebar"] h1 {
              font-size: 1.5rem !important;
              margin-bottom: 1.5rem !important;
              padding-bottom: 0.5rem !important;
              border-bottom: 2px solid #4ECDC4 !important;
          }
      </style>
      """, unsafe_allow_html=True)

    if 'current_page' not in st.session_state:
        st.session_state.current_page = "키워드 분석"

    pages = {
        "주요 토픽": "menu_keywords",
        "충전소 현황": "menu_heatmap_new",
        "상관분석결과": "menu_correlation",
        "인프라 현황": "menu_infrastructure",
        "분석 리포트": "menu_conclusion",
        "FAQ" : "menu_FAQ"
    }

    for page, key in pages.items():
        if st.sidebar.button(page, key=key):
            st.session_state.current_page = page

# --- Plotting Functions ---

def create_pie_chart(data, title):
    """파이 차트 생성"""
    fig, ax = plt.subplots(figsize=(8, 6))
    wedges, texts, autotexts = ax.pie(
        data['ev_counts'],
        labels=data['regions'],
        autopct='%1.1f%%',
        startangle=90,
        colors=data['colors']
    )
    ax.set_title(title, fontsize=14, pad=20)
    plt.tight_layout()
    return fig

def create_correlation_plot_regional(data):
    """지역별 상관관계 산포도 생성 (전기차 대수 vs 충전소 개수)"""
    fig, ax = plt.subplots(figsize=(10, 6))

    correlation = np.corrcoef(data['ev_counts'], data['charger_counts'])[0, 1]

    scatter = ax.scatter(data['ev_counts'], data['charger_counts'],
                             c=data['colors'], s=150, alpha=0.7, edgecolors='black')

    z = np.polyfit(data['ev_counts'], data['charger_counts'], 1)
    p = np.poly1d(z)
    ax.plot(data['ev_counts'], p(data['ev_counts']), "r--", alpha=0.8, linewidth=2)

    for i, region in enumerate(data['regions']):
        ax.annotate(region, (data['ev_counts'][i], data['charger_counts'][i]),
                             xytext=(5, 5), textcoords='offset points', fontsize=10)

    ax.set_xlabel('전기차 등록 대수')
    ax.set_ylabel('충전소 개수')
    ax.set_title(f'지역별 전기차 vs 충전소 상관관계 (r = {correlation:.3f})')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig, correlation

def create_time_series(data):
    """연도별 전기차 및 충전소 증가 추이 시계열 차트 생성"""
    fig, ax = plt.subplots(figsize=(10, 6))
    ax2 = ax.twinx()

    line1 = ax.plot(data['years'], data['total_ev_yearly'], 'b-o', linewidth=3, label='전기차 (천 대)')
    ax.set_ylabel('전기차 등록대수 (천 대)', color='blue')
    ax.tick_params(axis='y', labelcolor='blue')

    line2 = ax2.plot(data['years'], data['total_charger_yearly'], 'g-s', linewidth=3, label='충전소 (천 개)')
    ax2.set_ylabel('충전소 개수 (천 개)', color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    ax.set_xlabel('연도')
    ax.set_title('전기차 및 충전소 증가 추이')
    ax.grid(True, alpha=0.3)

    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc='upper left')
    plt.tight_layout()
    return fig

def plot_ev_per_charger_2024(df_merged_data):
    st.header("2024년 지역별 충전기당 전기차 수")
    df_2024 = df_merged_data[df_merged_data['year'] == 2024]
    if df_2024.empty:
        st.warning("2024년 데이터가 없습니다.")
        return

    df_bar = df_2024.groupby('name')['ev_per_charger'].mean().reset_index()
    df_bar = df_bar.sort_values(by='ev_per_charger', ascending=False)

    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(df_bar['name'], df_bar['ev_per_charger'], color='skyblue')

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.3, f'{height:.1f}',
                        ha='center', va='bottom', fontsize=9)

    ax.set_title("2024년 지역별 충전기당 전기차 수", fontsize=14)
    ax.set_xlabel("지역")
    ax.set_ylabel("전기차 대수 (충전기 1기당)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    st.pyplot(fig)
    plt.close(fig)

def plot_ev_per_charger_timeseries(df_merged_data):
    st.header("지역별 충전기당 전기차 수 변화 추이")
    df_grouped = df_merged_data.groupby(['full_date', 'name'])['ev_per_charger'].mean().reset_index()
    pivot = df_grouped.pivot(index='full_date', columns='name', values='ev_per_charger')

    fig, ax = plt.subplots(figsize=(14, 7))
    pivot.plot(marker='s', ax=ax)
    ax.set_title("지역별 충전기당 전기차 수", fontsize=14)
    ax.set_xlabel("날짜")
    ax.set_ylabel("전기차 대수 (충전기 1기당)")
    ax.legend(title="지역", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    st.pyplot(fig)
    plt.close(fig)

def plot_ev_ratio_timeseries(df_merged_data):
    st.header("지역별 전기차 비율 변화 추이")
    df_grouped = df_merged_data.groupby(['full_date', 'name'])['ev_ratio'].mean().reset_index()
    pivot = df_grouped.pivot(index='full_date', columns='name', values='ev_ratio')

    fig, ax = plt.subplots(figsize=(14, 7))
    pivot.plot(marker='o', ax=ax)
    ax.set_title("지역별 전기차 비율 변화 (%)", fontsize=14)
    ax.set_xlabel("날짜")
    ax.set_ylabel("전기차 비율 (%)")
    ax.legend(title="지역", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    st.pyplot(fig)
    plt.close(fig)

def plot_car_charger_correlation_national(df_car_data, df_charger_data):
    st.header("전국 전기차 수와 충전기 수 상관관계")
    df_car_corr = df_car_data.copy()
    df_charger_corr = df_charger_data.copy()

    df_car_corr['full_date'] = pd.to_datetime(df_car_corr['full_date'])
    df_charger_corr['full_date'] = pd.to_datetime(df_charger_corr['full_date'])

    car_total = df_car_corr.groupby('full_date')['car'].sum().reset_index()
    char_total = df_charger_corr.groupby('full_date')['total'].sum().reset_index()

    df_total = pd.merge(car_total, char_total, on='full_date')
    df_total['car'] = df_total['car'] / 1000
    df_total['total'] = df_total['total'] / 1000

    corr, p_value = pearsonr(df_total['car'], df_total['total'])

    st.write(f"**전체 기간 전기차 수와 충전기 수 피어슨 상관계수:** `{corr:.4f}` (유의확률: `{p_value:.4f}`)")

    def thousands(x, pos):
        return f'{int(x):,}'

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df_total['full_date'], df_total['car'], marker='o', label='전기차 수 (천 대)')
    ax.plot(df_total['full_date'], df_total['total'], marker='x', linestyle='--', label='충전기 수 (천 대)')
    ax.set_xlabel("날짜")
    ax.set_ylabel("대수 (천 단위)")
    ax.set_title(f"전국 전기차 수 vs 충전기 수\n(피어슨 상관계수: {corr:.2f})", fontsize=14)
    ax.legend()
    ax.grid(True)
    ax.yaxis.set_major_formatter(FuncFormatter(thousands))

    plt.tight_layout()
    st.pyplot(fig)
    plt.close(fig)

    st.subheader("변화량 기반 상관관계 분석")
    df_total['car_diff'] = df_total['car'].diff()
    df_total['total_diff'] = df_total['total'].diff()
    df_diff = df_total.dropna(subset=['car_diff', 'total_diff'])

    if not df_diff.empty:
        corr_diff, p_value_diff = pearsonr(df_diff['car_diff'], df_diff['total_diff'])
        st.write(f"**월별 증가량 기반 피어슨 상관계수:** `{corr_diff:.4f}` (유의확률: `{p_value_diff:.4f}`)")
    else:
        st.write("데이터 부족으로 변화량 기반 상관관계를 계산할 수 없습니다.")


# --- Main App Logic ---
def main():
    # --- 한글 폰트 설정 ---
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if 'font_installed' not in st.session_state:
        try:
            fm.findfont('NanumGothic', fallback_to_default=False)
            plt.rcParams['font.family'] = 'NanumGothic'
            plt.rcParams['axes.unicode_minus'] = False
            st.session_state.font_installed = True
            print("NanumGothic font already configured.")
        except ValueError:
            print("Installing NanumGothic font...")
            try:
                st.info("나눔고딕 폰트를 설치 중입니다. (Colab 환경)")
                # Use subprocess.run for shell commands
                subprocess.run(['apt-get', 'update', '-qq'], check=True)
                subprocess.run(['apt-get', 'install', 'fonts-nanum', '-qq'], check=True)
                subprocess.run(['fc-cache', '-fv'], check=True)
                subprocess.run(['rm', '-rf', '~/.cache/matplotlib'], check=True)

                plt.rcParams['font.family'] = 'NanumGothic'
                plt.rcParams['axes.unicode_minus'] = False
                st.session_state.font_installed = True
                st.success("나눔고딕 폰트 설치 및 설정 완료.")
            except subprocess.CalledProcessError as e:
                st.warning(f"나눔고딕 폰트 설치 오류 (쉘 명령 실행 실패): {e}. 그래프에 한글이 올바르게 표시되지 않을 수 있습니다.")
                plt.rcParams['font.family'] = 'DejaVu Sans'
                plt.rcParams['axes.unicode_minus'] = False
                st.session_state.font_installed = False
            except Exception as e:
                st.warning(f"나눔고딕 폰트 설치 또는 설정 오류: {e}. 그래프에 한글이 올바르게 표시되지 않을 수 있습니다.")
                plt.rcParams['font.family'] = 'DejaVu Sans'
                plt.rcParams['axes.unicode_minus'] = False
                st.session_state.font_installed = False
        except Exception as e:
            st.warning(f"폰트 설정 확인 중 오류 발생: {e}. 기본 폰트로 표시됩니다.")
            plt.rcParams['font.family'] = 'DejaVu Sans'
            plt.rcParams['axes.unicode_minus'] = False
            st.session_state.font_installed = False


    with st.spinner("필수 데이터를 로드 중입니다..."):
        common_data = load_common_data()
    with st.spinner("충전소 위치 데이터를 로드 중입니다..."):
        charger_location_df = load_charger_location_data()

    render_sidebar()

    if st.session_state.current_page == "충전소 현황":
        st.title("전기차 충전소 현황 대시보드")
        st.markdown("---")

        col_main_map, col_right_panel_map = st.columns([4, 1])

        with col_right_panel_map:
            st.subheader("지도 설정")

            if not charger_location_df.empty:
                min_year = int(charger_location_df['설치년도'].min())
                max_year = int(charger_location_df['설치년도'].max())
            else:
                min_year = 2010
                max_year = 2024

            # st.markdown("충전소 설치 연도")
            year_range = st.slider(
                "설치 연도",
                min_value=min_year,
                max_value=max_year,
                value=(min_year, max_year),
                step=1,
                key="map_year_slider",
                label_visibility="hidden"
            )
            st.markdown("")
            # st.markdown("지역 선택")
            if not charger_location_df.empty and '시도' in charger_location_df.columns:
                regions = ['전체'] + sorted(charger_location_df['시도'].unique().tolist())
            else:
                regions = ['전체']
            selected_region = st.selectbox("시도 선택", regions, key="map_region_select")
            st.markdown("")

            # st.markdown("지도 옵션")
            show_heatmap = st.checkbox("히트맵 표시", value=True, key="map_show_heatmap")
            show_markers = st.checkbox("마커 표시", value=False, key="map_show_markers")

            # st.markdown("히트맵 데이터 개수")
            st.markdown("")

            heatmap_limit = 5000
            if show_heatmap:
                heatmap_limit = st.slider(
                    "충전소 수",
                    min_value=1000,
                    max_value=min(20000, len(charger_location_df) if not charger_location_df.empty else 10000),
                    value=min(5000, len(charger_location_df) if not charger_location_df.empty else 5000),
                    step=1000,
                    help="더 적은 수로 설정하면 속도가 향상됩니다",
                    key="heatmap_limit_slider",
                    # label_visibility="hidden"
                )

            marker_limit = 500
            if show_markers:
                marker_limit = st.slider(
                    "충전소 수",
                    min_value=100,
                    max_value=min(5000, len(charger_location_df) if not charger_location_df.empty else 2000),
                    value=min(500, len(charger_location_df) if not charger_location_df.empty else 500),
                    step=100,
                    help="너무 많은 마커는 성능을 저하시킵니다",
                    key="marker_limit_slider",
                    # label_visibility="hidden"
                )


        with col_main_map:
            col_map_stats_1, col_map_stats_2, col_map_stats_3 = st.columns(3)

            @st.cache_data
            def filter_map_data(df, year_start, year_end, region):
                filtered_df = df[
                    (df['설치년도'] >= year_start) &
                    (df['설치년도'] <= year_end)
                ]
                if region != '전체':
                    filtered_df = filtered_df[filtered_df['시도'] == region]
                return filtered_df

            filtered_charger_df = filter_map_data(charger_location_df, year_range[0], year_range[1], selected_region)


            with col_map_stats_1:
                st.metric("충전소 수", f"{len(filtered_charger_df):,}개")

            with col_map_stats_2:
                regions_count_map = filtered_charger_df['시도'].nunique() if '시도' in filtered_charger_df.columns else 0
                st.metric("지역", f"{regions_count_map}개")

            with col_map_stats_3:
                if year_range[0] != year_range[1]:
                    years_count_map = year_range[1] - year_range[0] + 1
                    st.metric("선택 기간", f"{years_count_map}년")
                else:
                    st.metric("선택 연도", f"{year_range[0]}년")

            st.markdown("---")

            if not filtered_charger_df.empty:
                if selected_region != '전체':
                    center_lat = filtered_charger_df['위도'].mean()
                    center_lon = filtered_charger_df['경도'].mean()
                    zoom_start = 9
                else:
                    center_lat = 36.5
                    center_lon = 127.5
                    zoom_start = 7

                map_start_time = time.time()

                with st.spinner("지도를 생성하는 중..."):
                    m = folium.Map(
                        location=[center_lat, center_lon],
                        zoom_start=zoom_start,
                        tiles='OpenStreetMap'
                    )

                    if show_heatmap:
                        heat_sample = filtered_charger_df.sample(n=min(len(filtered_charger_df), heatmap_limit), random_state=42)
                        heat_data = heat_sample[['위도', '경도']].values.tolist()

                        plugins.HeatMap(
                            heat_data,
                            radius=15,
                            blur=15,
                            max_zoom=13,
                            gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}
                        ).add_to(m)
                        if len(filtered_charger_df) > heatmap_limit:
                            st.info(f"성능 최적화를 위해 히트맵에 {heatmap_limit:,}개 지점만 표시됩니다.")


                    if show_markers:
                        marker_cluster = plugins.MarkerCluster(
                            maxClusterRadius=50,
                            disableClusteringAtZoom=10
                        ).add_to(m)

                        marker_sample = filtered_charger_df.sample(n=min(len(filtered_charger_df), marker_limit), random_state=42)

                        for idx, row in marker_sample.iterrows():
                            folium.Marker(
                                location=[row['위도'], row['경도']],
                                popup=folium.Popup(
                                    f"<b>{row['충전소명']}</b><br>{row['주소']}<br>설치년도: {int(row['설치년도'])}",
                                    max_width=300
                                ),
                                icon=folium.Icon(color='blue', icon='plug', prefix='fa')
                            ).add_to(marker_cluster)
                        if len(filtered_charger_df) > marker_limit:
                            st.info(f"성능 최적화를 위해 마커는 {marker_limit:,}개만 표시됩니다.")


                map_time = time.time() - map_start_time

                tab1, tab2, tab3 = st.tabs(["충전소 지도", "연도별 분석", "시도별 분석"])

                with tab1:
                    st.subheader("충전소 위치 지도")
                    st.markdown("")
                    # st.caption(f"지도 생성 시간: {map_time:.2f}초")
                    st_folium(m, width=900, height=600, returned_objects=["last_object_clicked"])

                with tab2:
                    if not filtered_charger_df.empty:
                        st.subheader("연도별 충전소 설치 현황")
                        st.markdown("")
                        year_trend = filtered_charger_df['설치년도'].value_counts().sort_index()
                        year_trend_df = year_trend.reset_index()
                        year_trend_df.columns = ['설치년도', '충전소 수']
                        # st.line_chart(year_trend_df.set_index('설치년도'))
                        fig, ax = plt.subplots(figsize=(6,4))
                        ax.plot(year_trend_df['설치년도'], year_trend_df['충전소 수'],
                                marker='o', linewidth=2, markersize=6)
                        ax.set_xlabel('설치년도')
                        ax.set_ylabel('충전소 수')
                        ax.grid(True, alpha=0.3)
                        plt.tight_layout()
                        st.pyplot(fig, use_container_width=False)
                        plt.close(fig)
                    else:
                        st.warning("시도별 충전소 현황을 표시할 지역 정보가 없습니다.")

                with tab3:
                    # st.subheader("상세 분석 보기")
                    if not filtered_charger_df.empty:
                        st.subheader("시도별 충전소 현황 (TOP 10)")
                        st.markdown("")
                        if '시도' in filtered_charger_df.columns:
                            region_chart = filtered_charger_df['시도'].value_counts().head(10)

                            fig, ax = plt.subplots(figsize=(6,4))

                            bars = ax.bar(region_chart.index, region_chart.values,
                                            color='skyblue', edgecolor='navy', alpha=0.7)

                            ax.set_xlabel('시도')
                            ax.set_ylabel('충전소 수')
                            ax.set_title('시도별 충전소 현황')

                            for bar in bars:
                                height = bar.get_height()
                                ax.text(bar.get_x() + bar.get_width()/2., height,
                                        f'{int(height)}',
                                        ha='center', va='bottom')

                            plt.xticks(rotation=0)
                            plt.tight_layout()
                            st.pyplot(fig, use_container_width=False)
                            plt.close(fig)
                        else:
                            st.warning("시도별 충전소 현황을 표시할 지역 정보가 없습니다.")

            else:
                st.warning("선택한 조건에 해당하는 충전소가 없습니다. 필터 조건을 조정해주세요.")


    elif st.session_state.current_page == "주요 토픽":
        st.header("커뮤니티 내 전기차 관련 주요 키워드 분석")
        st.write("**2020년 ~ 2024년, 보배드림 게시글**")
        st.markdown("---")
        st.markdown("""
        <div style="text-align: right; margin-top: -20px; margin-bottom: 20px;">
            <small style="color: #666;">
                출처 : 2020년 ~ 2024년, 보배드림
            </small>
        </div>
        """, unsafe_allow_html=True)

        if not common_data['word_count']:
            st.warning("워드클라우드를 생성할 키워드 데이터가 비어있습니다. 'ele_crawling.csv' 파일과 내용을 확인해주세요.")
        else:

            col1, col2, col3 = st.columns(3)
            with col1:
                top_word = common_data['word_count'].most_common(1)[0][0]
                st.metric(label="인기 단어", value=top_word, delta="1위")
            with col2:
                top_count = common_data['word_count'].most_common(1)[0][1]
                st.metric(label="언급 수", value=f"{top_count}번", delta="최고")
            with col3:
                total_keywords = len(common_data['word_count'])
                st.metric(label="총 단어", value=f"{total_keywords}개", delta="고유 단어")

            try:
                def create_circle_mask(size):
                    y, x = np.ogrid[:size, :size]
                    center = size / 2
                    mask = (x - center) ** 2 + (y - center) ** 2 > (size/2) ** 2
                    return mask.astype(int) * 255

                mask = create_circle_mask(600)

                wordcloud = WordCloud(width=1200,
                                      height=800,
                                      background_color='white',
                                      font_path=font_path,
                                      mask=mask,
                                      collocations=False,
                                      colormap='YlGnBu',
                                      min_font_size=15,
                                      max_font_size=100,
                                      max_words=150,
                                      relative_scaling=0.4
                                      ).generate_from_frequencies(common_data['word_count'])
                fig, ax = plt.subplots(figsize=(5,5), dpi=300)
                ax.imshow(wordcloud, interpolation='bilinear')
                ax.axis('off')
                col_center = st.columns([1, 1.5, 1])
                with col_center[1]:
                    st.pyplot(fig, dpi=300)
                plt.close(fig)
            except Exception as e:
                st.error(f"워드클라우드 생성 오류: {e}. 'konlpy' 설치 및 한글 폰트(나눔고딕) 설정을 확인해주세요.")

    elif st.session_state.current_page == "인프라 현황":
        st.header("인프라 현황 분석")

        tab1, tab2, tab3, tab4 = st.tabs(["(2024년) 지역별 전기차 분포", "지역별 EV 비율", "(2024년) 지역별 충전기당 EV", "지역별 충전기당 EV"])

        with tab1:
            st.subheader("지역별 전기차 및 충전소 게이지도넛")

            diagnosis_df = common_data['regional_summary'].copy()
            diagnosis_df.rename(columns={
                'name': '지역',
                'ev_counts': '전기차등록수',
                'charger_counts': '충전기수'
            }, inplace=True)

            ev_list = diagnosis_df["전기차등록수"].tolist()
            ch_list = diagnosis_df["충전기수"].tolist()
            viz_data = list(zip(diagnosis_df["지역"], ev_list, ch_list))

            max_ev = max(ev_list) * 1.0
            max_ch = max(ch_list) * 1.0

            fig, axs = plt.subplots(1, len(viz_data), figsize=(len(viz_data)*2.6, 3.5), constrained_layout=True)
            if len(viz_data) == 1:
                axs = [axs]

            for ax, (region, ev, ch) in zip(axs, viz_data):
                ax.set_aspect("equal")
                ax.set_xlim(-1.2, 1.2)
                ax.set_ylim(-1.2, 1.2)
                ax.axis("off")

                ev_theta = (ev / max_ev) * 270
                ch_theta = (ch / max_ch) * 270

                ax.add_patch(patches.Wedge(
                    center=(0, 0), r=1.0, theta1=135, theta2=135 + ev_theta,
                    width=0.22, facecolor="#4F81BD", alpha=0.9
                ))
                ax.add_patch(patches.Wedge(
                    center=(0, 0), r=0.7, theta1=135, theta2=135 + ch_theta,
                    width=0.22, facecolor="#C0504D", alpha=0.85
                ))

                ax.text(0, -1.1, region, ha='center', fontsize=10, weight='bold')
                ax.text(0, -1.3, f"EV: {ev:,}", ha='center', fontsize=9, color="#4F81BD")
                ax.text(0, -1.45, f"충전소: {ch:,}", ha='center', fontsize=9, color="#C0504D")

            st.pyplot(fig)
            plt.close(fig)

            df_regional_display = common_data['regional_summary'].copy()
            df_regional_display['전기차당 충전소'] = np.round(df_regional_display['charger_counts'] / df_regional_display['ev_counts'], 3)
            df_regional_display.rename(columns={
                'name': '지역',
                'ev_counts': '전기차 (대)',
                'charger_counts': '충전소 (개)',
                'avg_ev_per_charger': '충전기 1기당 EV (평균)'
            }, inplace=True)
            st.dataframe(df_regional_display, use_container_width=True)


        with tab3:
            plot_ev_per_charger_2024(common_data['df_ev_chg_merge'])

        with tab4:
            plot_ev_per_charger_timeseries(common_data['df_ev_chg_merge'])

            # '경기'를 제외한 데이터만 필터링
            filtered_df = common_data['df_ev_chg_merge'][common_data['df_ev_chg_merge']['name'] != '경기']
            # 필터링된 데이터로 그래프 출력
            st.markdown("---")
            st.subheader("특이값(경기) 제외")
            plot_ev_per_charger_timeseries(filtered_df)


        with tab2:
            plot_ev_ratio_timeseries(common_data['df_ev_allcar_merge'])

            # '제주'를 제외한 데이터만 필터링
            filtered_df = common_data['df_ev_allcar_merge'][common_data['df_ev_allcar_merge']['name'] != '제주']
            # 필터링된 데이터로 그래프 출력
            st.markdown("---")
            st.subheader("특이값(제주) 제외")
            plot_ev_ratio_timeseries(filtered_df)




    elif st.session_state.current_page == "상관분석결과":
        st.header("상관관계 분석")
        plot_car_charger_correlation_national(common_data['df_car'], common_data['df_charger'])
        st.markdown("---")
        st.subheader("지역별 전기차 vs 충전소 상관관계 (최신 데이터)")
        fig, correlation_regional = create_correlation_plot_regional(common_data)
        st.pyplot(fig)
        plt.close(fig)
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("상관계수 (지역별)", f"{correlation_regional:.3f}")
        with col2:
            st.metric("결정계수 (지역별)", f"{correlation_regional**2:.3f}")
        with col3:
            st.metric("해석", "강한 양의 상관관계" if correlation_regional > 0.7 else ("음의 상관관계" if correlation_regional < -0.3 else "약한 상관관계"))

        st.markdown("---")



    elif st.session_state.current_page == "분석 리포트":
        st.header("충전스트레스")
        st.write("각 지역의 전기차 수와 충전소 수를 비교하여 충전 스트레스 정도를 측정합니다.")

        scores = []
        recommendations = []
        if not common_data['regional_summary'].empty:
            max_ev_per_charger_val = common_data['regional_summary']['avg_ev_per_charger'].max()
            min_ev_per_charger_val = common_data['regional_summary']['avg_ev_per_charger'].min()

            for idx, row in common_data['regional_summary'].iterrows():
                ev_per_charger = row['avg_ev_per_charger']
                if pd.isna(ev_per_charger) or ev_per_charger == 0:
                    score = 0
                elif max_ev_per_charger_val == min_ev_per_charger_val:
                    score = 50
                else:
                    score = ((max_ev_per_charger_val - ev_per_charger) / (max_ev_per_charger_val - min_ev_per_charger_val)) * 100

                scores.append(round(score))

                if score >= 70:
                    recommendations.append("🟢 낮음 ")
                elif score >= 50:
                    recommendations.append("🟡 보통 ")
                elif score >= 30:
                    recommendations.append("🟠 높음 ")
                else:
                    recommendations.append("🔴 매우 높음 ")

        result_df = pd.DataFrame({
            '지역': common_data['regions'],
            '전기차 수': common_data['ev_counts'],
            '충전소 수': common_data['charger_counts'],
            '충전기 1기당 전기차': common_data['regional_summary']['avg_ev_per_charger'].round(1).tolist(),
            '인프라 점수': scores,
            '충전 스트레스': recommendations
         })

        st.dataframe(result_df.sort_values(by='인프라 점수', ascending=False),
                     use_container_width=True,
                     hide_index=True
                     )
        st.markdown("")
        fig, ax = plt.subplots(figsize=(10, 6))

        # 데이터 정렬
        sorted_df = result_df.sort_values(by='인프라 점수', ascending=True)  # 차트용으로 오름차순

        # 바 차트 그리기
        bars = ax.bar(sorted_df['지역'], sorted_df['인프라 점수'],
                      color='skyblue', edgecolor='navy', alpha=0.7)

        # 각 바 위에 점수 표시
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{int(height)}',
                    ha='center', va='bottom', fontweight='bold')

        ax.set_xlabel('지역')
        ax.set_ylabel('인프라 점수')
        ax.set_title('지역별 인프라 점수')
        ax.grid(True, alpha=0.3, axis='y')

        # 🔧 x축 글자 가로로 설정
        plt.xticks(rotation=0)  # 0도 = 가로
        plt.tight_layout()

        st.pyplot(fig)
        plt.close(fig)
         # st.info("💡 추천 점수는 해당 지역의 전기차 수 대비 충전소 수를 기반으로 계산됩니다. 점수가 높을수록 전기차 충전 인프라 환경이 더 나쁘다고 해석할 수 있습니다.")

    elif st.session_state.current_page == "FAQ":
      st.header("전기차 FAQ 검색")

      col1, col2 = st.columns([4, 1])

      with col1:
          search_query = st.text_input(
              "검색어 입력",
              label_visibility="collapsed"
          )
      with col2:
          search_button = st.button("검색", type="primary", use_container_width=True)

      # 🔹 2. 파일 경로 지정
      csv_path = "/content/crawling_car.csv"
      df = pd.read_csv(csv_path, encoding='cp949')  # 질문, 답변 컬럼 존재 가정

      # 🔹 3. 랜덤 날짜 생성 함수
      def random_date(start, end):
          delta = end - start
          random_days = random.randint(0, delta.days)
          return (start + timedelta(days=random_days)).strftime('%Y-%m-%d')

      # 🔹 4. 날짜 범위 및 출처 리스트 설정
      start_date = datetime.strptime("2023-01-01", "%Y-%m-%d")
      end_date = datetime.strptime("2024-12-31", "%Y-%m-%d")
      sources = ["기아", "현대", "테슬라"]

      # 🔹 5. FAQ 데이터 리스트 생성
      faq_data = []
      for _, row in df.iterrows():
          item = {
              "category": "전기차",
              "question": row["question"],
              "answer": row["answer"],
              "source": random.choice(sources),
              "date": random_date(start_date, end_date),
              "views": random.randint(100, 5000)
          }
          faq_data.append(item)

      # 🔹 6. 검색 결과 필터링
      if search_query:
          filtered_faq = [
              faq for faq in faq_data
              if search_query.lower() in faq['question'].lower() or
                search_query.lower() in faq['answer'].lower() or
                search_query.lower() in faq['category'].lower()
          ]
      else:
          filtered_faq = faq_data

      # 🔹 7. 검색 결과 개수 표시
      st.metric("검색 결과", f"{len(filtered_faq)}건")
      st.markdown("---")

      # 🔹 8. FAQ 출력
      if len(filtered_faq) == 0:
          st.info("🔍 검색 결과가 없습니다. 다른 검색어를 입력해보세요.")
      else:
          for i in range(0, len(filtered_faq), 2):
              cols = st.columns(2)
              for j in range(2):
                  faq_index = i + j  # 🔑 유일한 인덱스로 키 생성
                  if faq_index < len(filtered_faq):
                      faq = filtered_faq[faq_index]
                      with cols[j]:
                          with st.container():
                              st.markdown(
                                  f"""
                                  <div style="
                                      background-color: #f8f9fa;
                                      padding: 20px;
                                      border-radius: 10px;
                                      border-left: 4px solid #4ECDC4;
                                      margin-bottom: 10px;
                                      height: 220px;
                                  ">
                                      <span style="
                                          background-color: #4ECDC4;
                                          color: white;
                                          padding: 2px 8px;
                                          border-radius: 4px;
                                          font-size: 12px;
                                      ">{faq['category']}</span>
                                      <h4 style="margin-top: 10px; color: #2c3e50; font-size: 16px;">
                                          {faq['question']}
                                      </h4>
                                      <p style="color: #555; margin: 10px 0; font-size: 14px;">
                                          {str(faq['answer'])[:100]}...
                                      </p>
                                      <div style="
                                          display: flex;
                                          justify-content: space-between;
                                          align-items: center;
                                          margin-top: 15px;
                                          font-size: 12px;
                                          color: #888;
                                      ">
                                          <span>📰 {faq['source']}</span>
                                          <span>👁️ {faq['views']:,}회</span>
                                      </div>
                                  </div>
                                  """,
                                  unsafe_allow_html=True
                              )

                              if st.button("자세히 보기", key=f"detail_{faq_index}", use_container_width=True):
                                  with st.expander("전체 답변", expanded=True):
                                      st.write(f"**Q. {faq['question']}**")
                                      st.write(faq['answer'])
                                      st.caption(f"출처: {faq['source']} | 작성일: {faq['date']}")


    st.markdown("---")

if __name__ == "__main__":
    main()

Overwriting app.py


In [14]:
!wget -q -O - ipv4.icanhazip.com

34.127.60.3


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.60.3:8501

⠦⠧⠇⠏⠋your url is: https://curvy-walls-see.loca.lt
NanumGothic font already configured.
